In [8]:
%load_ext autotime

In [9]:
from selenium import webdriver

from tqdm import tqdm
import time
import os
import glob
import datetime
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [10]:
pswd = '8mzcr9rm7tmx'
usrn = 'nq9ky5k48'

In [11]:
url="https://vip.netvideogirls.com/members/"

# drivesL = ["H:\\NVG", "Y:\\NVG2","E:\\NVG3","I:\\NVG4"]
drivesL = ["N:\\VIDEO_APP\\assets"]

img_path = r"N:/VIDEO_APP/assets/imgs"
download_path = drivesL[0]
remote_path = r"N:/VIDEO_APP"

In [12]:
options = webdriver.EdgeOptions()

prefs = {
      "download.default_directory": download_path,
      "download.directory_upgrade": True,
      # "download.prompt_for_download": False,
}

options.add_experimental_option('prefs', prefs)
options.use_chromium = True
# options.add_argument('headless')
options.add_argument('disable-gpu')

edge_path = remote_path+'/edgedriver_win64/msedgedriver.exe'

driver = webdriver.Edge(executable_path=edge_path,
                          options=options)

In [13]:
def login(url,usernameId='username', username=usrn, passwordId='password', pswd=pswd, submit_buttonId='Sign In'):
    driver.get(url)
    driver.find_element_by_id(usernameId).send_keys(username)
    driver.find_element_by_id(passwordId).send_keys(pswd)
    driver.find_element_by_class_name('btn').click()
login(url="https://vip.netvideogirls.com/members/")

In [14]:
def with_moviepy(filename):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(filename)
    duration       = clip.duration
    fps            = clip.fps
    width, height  = clip.size
    return duration, fps, width, height

In [15]:
vid_df = pd.DataFrame([], columns=['vid_id', 'url', 'g_id', 'g_name', 'tag_id', 'tag_name', 'date', 'year','cite',
                                        'vid_file', 'base_file', 'file_url','duration', 'fps', 'width', 'height'])


for vid_no in tqdm(range(2250, 2300)):
    
    ############ Skip If Already Exists ############
    video_nos = []
    for d_path in drivesL:
        video_nos += [int(re.findall(string=file, pattern=r'\\([0-9]+)-.*.mp4')[0]) for file in \
                 glob.glob(d_path+ '/*/*.mp4')]
    
    if vid_no in video_nos:
        continue
    
    ############ GO TO VIDEO ############
    url = "https://vip.netvideogirls.com/members/video/{}".format(vid_no)  
    driver.get(url) 
    print(url)
    
    # Wait 5 seconds for page to load
    time.sleep(5)
    
    ########### OPEN DROPDOWN ############
    try:
        driver.find_element_by_class_name('icon-download').click()
    except:
        print("File Not Available")
        continue
    
    ########### Download File ############
    item_els = driver.find_elements_by_class_name('item')
    hd_link = [x for x in item_els if 'HD' in x.text]
    sd_link = [x for x in item_els if 'SD' in x.text]
    
    if len(hd_link) > 0:
        hd_link[0].click()
    elif len(sd_link) > 0:
        sd_link[0].click()
    else:
        print('No MP4 file')
        break
    
    ############ GET YEAR ############
    spans = driver.find_elements_by_tag_name('span')
    for span in spans:

        date_str = span.text
        try:
            date = datetime.datetime.strptime(date_str, '%b %d, %Y')
            year = date.year
            # print(year)
        except:
            continue
    
    ############ GET LINKS ############
    linkL = driver.find_elements_by_tag_name('a')

    ############ GET NAMES ############
    namesL = driver.find_elements_by_class_name('list_box')[0].text.split('\n')[1:]
    g_linkL = [link for link in linkL if 'https://vip.netvideogirls.com/members/girl/' in link.get_attribute("href")][:len(namesL)]
    hrefL = [link.get_attribute("href") for link in g_linkL]
    g_dict = {link.split('/')[-1]:link.split('/')[-2] for link in hrefL}

    ############ GET CITE ############
    cite = driver.find_elements_by_class_name('subname')[0].find_element_by_tag_name('a').text

    ############ GET TAGS ############
    tagL = driver.find_elements_by_class_name('tag_box')[0].find_elements_by_tag_name('a')
    tag_linkL = [link.get_attribute("href") for link in tagL if '/members/tag/' in link.get_attribute("href")]
    tag_dict = {link.split('/')[-1]:link.split('/')[-2] for link in tag_linkL}

    #################################
    ######### GET IMG POSTER ########
    #################################
    file_name = 'vid_posters/vid-' + str(vid_no) + '-poster.jpg'
    full_file_name = "../assets/imgs/" + file_name

    ############ GO TO IMAGE ############
    img_url = driver.find_elements_by_class_name("react-player__preview")[0].value_of_css_property('background-image').lstrip('url("').rstrip('")')
    driver.get(img_url)
    time.sleep(5)
    imgs = driver.find_elements_by_tag_name('img')

    ############ TAKE SCREENSHOT ############
    imgs[0].screenshot(full_file_name)

    #################################
    ############ GET G POSTER #######
    #################################
    for g_id, g_name in g_dict.items():
        url = "https://vip.netvideogirls.com/members/girl/" + g_name + "/" + str(g_id)
        driver.get(url)
        time.sleep(2)

        ############ GET G IMG NAME ############
        file_name =  'g_posters/g-' + g_id + '-poster.jpg'
        full_file_name = "../assets/imgs/" + file_name

        ############ GO TO IMAGE ############
        img_urlL = [x.find_element_by_tag_name('img').get_attribute('src') for x in driver.find_elements_by_class_name("avatar")]
        img_url = [x for x in img_urlL if str(g_id) in str(x)][0]
        
        driver.get(img_url)
        time.sleep(2)
        ########### FIND IMAGE SCREENSHOT ###
        imgs = driver.find_elements_by_tag_name('img')

        ############ TAKE SCREENSHOT ############
        imgs[0].screenshot(full_file_name)


    ########### Move to Folder ############
    files = glob.glob(download_path + '/*.mp4')
    file = [x for x in files if '\\' + str(vid_no)+'-' in x]
    
    while len(file) == 0:
        time.sleep(15)
        files = glob.glob(download_path + '/*.mp4')
        file = [x for x in files if '\\' + str(vid_no)+'-' in x]
    file = file[0]
    
    new_folder = download_path+'/'+str(year)
    new_file = download_path+'/'+str(year)+'/'+ os.path.basename(file)
    
    try:
        os.mkdir(download_path+'/'+str(year))
    except:
        pass
    os.replace(file, new_file)

    ############# GET VIDEO INFO ##############
    video_info = with_moviepy(new_file)

    ######### ADD TO DATAFRAME #########
    vid_df = pd.DataFrame([], columns=['vid_id', 'url', 'g_id', 'g_name', 'tag_id', 'tag_name', 'date', 'year','cite',
                                        'vid_file', 'base_file', 'file_url','duration', 'fps', 'width', 'height'])
    for g_id, g_name in g_dict.items():
        for tag_id, tag_name in tag_dict.items():
            vid_df = pd.concat([vid_df, pd.DataFrame.from_dict({'vid_id':vid_no, 
                                                                  'url':url, 
                                                                  'g_id':g_id, 
                                                                  'g_name':g_name,
                                                                  'tag_id':tag_id,
                                                                  'tag_name':tag_name, 
                                                                  'date':date, 
                                                                  'year':year,
                                                                  'cite':cite,
                                                                  'vid_file':file.replace(os.path.basename(file), '{year}\\'.format(year=str(year)) + os.path.basename(file)),
                                                                  'base_file':str(year)+'/'+os.path.basename(file),
                                                                  'file_url':'assets/'+str(year)+'/'+os.path.basename(file),
                                                                  'duration':video_info[0],
                                                                  'fps':video_info[1],
                                                                  'width':video_info[2],
                                                                  'height':video_info[3]}, orient='index'
                                                                ).T]).reset_index(drop=True)
            
        vid_df.to_csv('../info_files/vid_info_all.csv', mode='a', index=False, header=False)



    


  8%|▊         | 4/50 [00:00<00:00, 46.47it/s]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=103.0.1264.37)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::unpack [0x00007FF76D0F3712+24578]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x00007FF76D02E282+563682]
	Ordinal0 [0x00007FF76CB30FB5+659381]
	Ordinal0 [0x00007FF76CB10FBB+528315]
	Ordinal0 [0x00007FF76CB8A314+1024788]
	Ordinal0 [0x00007FF76CB9B0E2+1093858]
	Ordinal0 [0x00007FF76CB875C3+1013187]
	Ordinal0 [0x00007FF76CB5E4E0+845024]
	Ordinal0 [0x00007FF76CB5F8D8+850136]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF76CDE4EB8+119992]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF76CDD09C7+36807]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF76CDD3B5C+49500]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF76CC2F606+20582]
	Microsoft::App